In [1]:
import pandas as pd
import numpy as np
import re, os, pickle
from os.path import join
from tqdm import tqdm
from datetime import datetime

# 옛날거 반영해야 하므로 전산에서 다운 받는다.
# 필요컬럼은 채무자키,계좌키,합의서체결일,채무구분
# 합의서체결일자 있는 것과 주채무만 남기고 나머지는 삭제
# 주민번호는 비식별처리되어있으며, 계좌별로 합의서체결일이 다를 수 있으므로 계좌키로 작업한다.

In [8]:
# 파일읽기 #####################################################################################################
wd = r"D:\3.자산\프로젝트\2023 전산데이터 클렌징 및 업무개편 프로젝트\시효"
credit = pd.read_excel(join(wd, "신용회복새창_20231027_1717.xlsx"), dtype=str)
account = pd.read_pickle(r"D:\3.자산\전산 dataset\솔림\231011\계좌조회새창_20231011_1819.pkl")[['채무자키','계좌키','시효완성일','시효연장사유','시효중단여부']]
################################################################################################################

In [9]:
# 신복에서 시효완성일 및 시효연장사유 작성
credit["시효완성일"] = (pd.to_numeric(credit.합의서체결일.str[:4]) + 5).astype(str) +credit.합의서체결일.str[4:]
credit["시효연장사유"] = "신용회복 합의서 체결:"+credit.합의서체결일

# 병합 후 시효수정할 것만 
merged = credit.merge(account, on=['채무자키','계좌키'], how='left')
시효수정 = merged.query('시효완성일_x > 시효완성일_y')
print("중복제거 전 : ", len(시효수정))

# 시효완성일 역순정렬 후 중복계좌 드랍하기
시효수정 = 시효수정.sort_values(['시효완성일_x'], ascending=False)
시효수정 = 시효수정.drop_duplicates(['계좌키'], keep='first')
print("중복제거 후 : ", len(시효수정))

In [26]:
시효수정.rename(columns={'시효완성일_x':'최초시효완성일', '시효연장사유_x':'최초시효완성일메모'}, inplace=True)
###########################################################################
시효수정.to_excel(join(wd, "231027 신용회복 합의서체결일반영 시효수정.xlsx"), index=False)
###########################################################################

#### 전산에서 종결건 제외 후 엑셀일괄수정 > 계좌정보